In [54]:
# https://www.kaggle.com/code/lizhecheng/balanced-weights-longer-training-higher-score
# https://www.kaggle.com/code/bhanupratapbiswas/llm-detect-ai-generated-text-02
# 抄这两篇然后根据自己的理解改了下，只有数据处理的部分, 还没搭模型
# 有空可以看一下这两篇开头写的
import torch
import random
import numpy as np
import regex as re
import pandas as pd
import torch.nn as nn
import language_tool_python

from transformers import To
from sklearn.feature_extraction.text import TfidfVectorizer


def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)


seed = 42
seed_everything(seed)
device = "cuda"

In [42]:
essay_data = pd.read_csv("train_essays.csv")
test_data = pd.read_csv("test_essays.csv")
train_data = pd.concat([daigt_data, essay_data])
prompts = pd.read_csv("train_prompts.csv")

# https://www.kaggle.com/datasets/thedrcat/daigt-v2-train-dataset/data
# https://www.kaggle.com/datasets/nbroad/persaude-corpus-2/
# daigt data 上面两网址有描述 有兴趣可以看 可能是因为官方提供的数据比较少 所以得用外部的数据来训练模型
daigt_data = pd.read_csv("train_v2_drcat_02.csv")

In [80]:
language_tool = language_tool_python.LanguageTool("en-US")  # 一个语法检查器
# pip install language-tool-python
# 机子要有java才能下载工具包 没有java或者不想装也可以手动下载 https://languagetool.org/download/LanguageTool-stable.zip
# 解压后放到~/.cache/language_tool_python/


# 想了解可以看 https://pypi.org/project/language-tool-python/
# 用语法检查器修改句子
def correct_sentence(sentence):
    return language_tool.correct(sentence)

In [13]:
not_persuade_data = daigt_data[
    daigt_data["source"] != "persuade_corpus"
]  # 直译：不具有说服性的的文章数据
# 我的理解是这些所谓的not_persuade_data应该是指不那么正式的文章，可能就是人与人之间交流的句子，然后一些网络上的夹着表情的话
# 建议自己拿一篇出来看一下，或者也可以翻译成非议论文
persuade_data = daigt_data[
    daigt_data["source"] == "persuade_corpus"
]  # 直译：具有说服性的文章数据 或者议论文

In [41]:
formal_characters = set(list("".join(persuade_data.text.to_list())))  # 具有说服性的文章里所用到的字符
all_characters = set(
    list("".join(not_persuade_data.text.to_list()))
)  # 不具有说服性的文章里所用到的字符
remove_list = "".join(all_characters - formal_characters)  # 两个集合拿差集 得到要除去的字符表

translation_table = str.maketrans("", "", remove_list)  # 把得到的字符表塞入这个函数，通过remove_chars函数


# 每个字符串出现这种字符就会删掉
def remove_chars(string):
    return string.translate(translation_table)

In [81]:
remove_list  # 要移除的字符表

'📷💰🌲💻🌎🐠📅🍋🏊🍷🙀💘🚫🌊💊🏳护🎉”️驶🎃有🏜🎣🌄选安🇷ê😷…😝🏖🤓🔜🥨🎠🤤禁😢🍁部🗳🧠🥛🚨唯🙌🏰😴😉📱🚑🤖😊🐒🌫🐆🗣ã🔥こり💦せр手🚴🏋🧀💇ん🤘🧘😤💸🥁🛣🧐💼🏙🦸道🥤💀😍\r😂🛍😩‘💃🧡🐕🏛🏨🍴’з–🚌🐾🐴🥩🥯🛀🕰🤫🦄🇺中📖🔋🤯🌧🏠和😠É😲都👏必😮🧚🎩🎧🌱🐳👥🏔🍞🛫🤕🥘👕👍🍝意🎥🇸响み的司🎭😔🧑�🥲🎯👻🥟👬🍮😘😹🐟💔法🎨と😋🍲👋😃🎸😭♂🌟🚂🤝🎬🎓🔬🏄🌽🏼🌳□🌮🎄须🚗🏈🌐🌃🧭🧹👮🥔者д🎅择😒す🌞п😆🤟🛠😬🍎📰🤜🎤😵💅集📞あ时🚔👨🐶🍟🥖🧦🤪所🛋🏦╯👩全🏢🍖🦁🌴👧🔮с该🛸🥭💭注🍄📺🛬\u200d😄💧👇🕹🌨🎵🚀🐝💚🕵🌅🥑📹📊🏯💥🧙🌿🥜🐸ü🚣🥶😖👯💖🎊🏡📸🚕一取🏽💬保─应🌯机🙋🇧🙈🐢🤣💉😕📉🇯😎ā💨🍜🚚ç🐱🤛力に🔧🎮а用е🌷🤢🧩🥗🌻🐭止🥳🦐😓📄🍰😁\u200b❄🎶驾🚭☹🔭—💜🏫🏻将🌸🔑路上🌭🧬🤩🧖🧽🍳が😻😱📧💕😡是🇵🍕📚🐬☀う🍣📈🤗合😌✨💡🛑🏏🙅在🥕🐰🙊🤷👀😅о😳🌈🦎🍓💆♀⚽😈🐦🐻💪🏕🌌💫🤔👂📦📝🎾👌🥪🏃🇪🍗🌠🇫í🙃🏀🌏👦😨🐧👪ま🍭は💤🍿÷🚪🥦😜⏰影🍽使🍔ち💯💁🏟🏞🙏🕒🎹📣🙄“🤦🏥🔍🎢完🤒🏆。🤞🕺🎈👫'

In [52]:
train_data["text"] = train_data["text"].apply(remove_chars)  # 移除非正式字符
train_data["text"] = train_data["text"].str.replace("\n", "")  # \n 换成 ''

test_data["text"] = test_data["text"].str.replace("\n", "")
test_data["text"] = test_data["text"].apply(remove_chars)  # 移除非正式字符
test_data["text"] = test_data["text"].apply(correct_sentence)  # 用语法检查器改句子

data = pd.concat([train_data["text"], test_data["text"]], axis=0)  # 训练集和测试集合并

In [55]:
# 把句子向量化 提取特征
# https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction
# 正则表达式：r'[^\W]+' 直接看例子
# "This is an example sentence with various punctuation and whitespace."
# 应用正则表达式后
# ['This', 'is', 'an', 'example', 'sentence', 'with', 'various', 'punctuation', 'and', 'whitespace']
vectorizer = TfidfVectorizer(
    ngram_range=(3, 5),
    tokenizer=lambda x: re.findall(r"[^\W]+", x),
    token_pattern=None,
    strip_accents="unicode",
)
vectorizer = vectorizer.fit(test_data["text"])

In [58]:
X = vectorizer.transform(data)

In [75]:
X.toarray()

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])